In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.tools as tls
import plotly.figure_factory as ff


from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import precision_score, recall_score, confusion_matrix,  roc_curve, precision_recall_curve, accuracy_score, roc_auc_score
import lightgbm as lgbm
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve,auc
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from yellowbrick.classifier import DiscriminationThreshold

import scipy.stats as ss
from scipy import interp
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [ ]:
data = pd.read_excel("Dataset_Yakutia.xlsx")

In [ ]:
data.head()

,gender,age,reg_region_nm,cnt_tr_all_3m,cnt_tr_top_up_3m,cnt_tr_cash_3m,cnt_tr_buy_3m,cnt_tr_mobile_3m,cnt_tr_oil_3m,cnt_tr_on_card_3m,...,avg_outstanding_amount_3m,cnt_dep_act,sum_dep_now,avg_dep_avg_balance_1month,max_dep_avg_balance_3month,app_vehicle_ind,app_position_type_nm,visit_purposes,qnt_months_from_last_visit,super_clust
0,0.0,52.688172,Ханты-Мансийский автономный округ - Югра,201.0,18.0,6.0,97.0,7.0,46.0,11.0,...,NaN,0.0,279.88,277.87,276.027609,0.0,,DCARD,10.0,"a. Супер-ЗП (6,15)"
1,0.0,26.940860,Республика Татарстан (Татарстан),9.0,3.0,0.0,0.0,0.0,0.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,,DCARD,15.0,c. Масс-ЗП закредитованные (12)
2,0.0,47.419355,Город Санкт-Петербург,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,0.625650,NaN,,NaN,NaN,"k. Без продукта (7,13,18)"
3,NaN,49.551075,Республика Татарстан (Татарстан),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POTREB,10.0,"d. Масс-ЗП без согласия БКИ (1,3)"
4,NaN,55.591398,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"h. Бывшие зарплатники (4,9,10,11)"


In [ ]:
threshold = 0.2
data = data.dropna(thresh=int(threshold * len(data.columns)))
data

,gender,age,reg_region_nm,cnt_tr_all_3m,cnt_tr_top_up_3m,cnt_tr_cash_3m,cnt_tr_buy_3m,cnt_tr_mobile_3m,cnt_tr_oil_3m,cnt_tr_on_card_3m,...,avg_outstanding_amount_3m,cnt_dep_act,sum_dep_now,avg_dep_avg_balance_1month,max_dep_avg_balance_3month,app_vehicle_ind,app_position_type_nm,visit_purposes,qnt_months_from_last_visit,super_clust
0,0.0,52.688172,Ханты-Мансийский автономный округ - Югра,201.0,18.0,6.0,97.0,7.0,46.0,11.0,...,NaN,0.0,279.88,277.87,276.027609,0.0,,DCARD,10.0,"a. Супер-ЗП (6,15)"
1,0.0,26.940860,Республика Татарстан (Татарстан),9.0,3.0,0.0,0.0,0.0,0.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,,DCARD,15.0,c. Масс-ЗП закредитованные (12)
2,0.0,47.419355,Город Санкт-Петербург,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,0.625650,NaN,,NaN,NaN,"k. Без продукта (7,13,18)"
3,NaN,49.551075,Республика Татарстан (Татарстан),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POTREB,10.0,"d. Масс-ЗП без согласия БКИ (1,3)"
4,NaN,55.591398,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"h. Бывшие зарплатники (4,9,10,11)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,1.0,39.357527,Республика Коми,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,"i. Бывшие заемщики (2,5,19)"
993,0.0,29.424731,Курская область,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,"k. Без продукта (7,13,18)"
995,1.0,23.083333,Воронежская область,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,"k. Без продукта (7,13,18)"
998,0.0,31.494624,Белгородская область,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,"h. Бывшие зарплатники (4,9,10,11)"


In [ ]:
data

,gender,age,reg_region_nm,cnt_tr_all_3m,cnt_tr_top_up_3m,cnt_tr_cash_3m,cnt_tr_buy_3m,cnt_tr_mobile_3m,cnt_tr_oil_3m,cnt_tr_on_card_3m,...,avg_outstanding_amount_3m,cnt_dep_act,sum_dep_now,avg_dep_avg_balance_1month,max_dep_avg_balance_3month,app_vehicle_ind,app_position_type_nm,visit_purposes,qnt_months_from_last_visit,super_clust
0,0.0,52.688172,Ханты-Мансийский автономный округ - Югра,201.0,18.0,6.0,97.0,7.0,46.0,11.0,...,NaN,0.0,279.88,277.87,276.027609,0.0,,DCARD,10.0,"a. Супер-ЗП (6,15)"
1,0.0,26.940860,Республика Татарстан (Татарстан),9.0,3.0,0.0,0.0,0.0,0.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,,DCARD,15.0,c. Масс-ЗП закредитованные (12)
2,0.0,47.419355,Город Санкт-Петербург,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,0.625650,NaN,,NaN,NaN,"k. Без продукта (7,13,18)"
3,NaN,49.551075,Республика Татарстан (Татарстан),0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POTREB,10.0,"d. Масс-ЗП без согласия БКИ (1,3)"
4,NaN,55.591398,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"h. Бывшие зарплатники (4,9,10,11)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,1.0,39.357527,Республика Коми,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,"i. Бывшие заемщики (2,5,19)"
993,0.0,29.424731,Курская область,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,"k. Без продукта (7,13,18)"
995,1.0,23.083333,Воронежская область,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,"k. Без продукта (7,13,18)"
998,0.0,31.494624,Белгородская область,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,"h. Бывшие зарплатники (4,9,10,11)"


In [ ]:
mean_age = data['age'].mean()
data['age'].fillna(mean_age, inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 859 entries, 0 to 999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   gender                      573 non-null    float64
 1   age                         859 non-null    float64
 2   reg_region_nm               843 non-null    object 
 3   cnt_tr_all_3m               477 non-null    float64
 4   cnt_tr_top_up_3m            477 non-null    float64
 5   cnt_tr_cash_3m              477 non-null    float64
 6   cnt_tr_buy_3m               477 non-null    float64
 7   cnt_tr_mobile_3m            477 non-null    float64
 8   cnt_tr_oil_3m               477 non-null    float64
 9   cnt_tr_on_card_3m           477 non-null    float64
 10  cnt_tr_service_3m           477 non-null    float64
 11  cnt_zp_12m                  213 non-null    float64
 12  sum_zp_12m                  213 non-null    float64
 13  limit_exchange_count        181 non

In [ ]:
data.super_clust.unique()

array(['a. Супер-ЗП (6,15)', 'c. Масс-ЗП закредитованные (12)',
       'k. Без продукта (7,13,18)', 'd. Масс-ЗП без согласия БКИ (1,3)',
       'h. Бывшие зарплатники (4,9,10,11)', 'j. Депозиты в оттоке (0,17)',
       'b. Текущие заемщики (14)', 'e. Супер-аффлуент (-1)',
       'g. ДК/ЗК до 6 моба (20)', 'f. Супер-депозиты (8,16)',
       'i. Бывшие заемщики (2,5,19)'], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data['reg_region_nm'] = label_encoder.fit_transform(data['reg_region_nm'])
data['visit_purposes'] = label_encoder.fit_transform(data['visit_purposes'])
data['app_position_type_nm'] = label_encoder.fit_transform(data['app_position_type_nm'])

In [ ]:
data['gender'].fillna(-1, inplace=True)

In [ ]:
columns = ['cnt_tr_all_3m', 'cnt_tr_top_up_3m', 'cnt_tr_cash_3m', 'cnt_tr_buy_3m', 'cnt_tr_mobile_3m', 'cnt_tr_oil_3m', 'cnt_tr_on_card_3m', 'cnt_tr_service_3m', 'cnt_zp_12m', 'sum_zp_12m', 'max_outstanding_amount_6m', 'avg_outstanding_amount_3m', 'cnt_dep_act', 'sum_dep_now', 'avg_dep_avg_balance_1month', 'max_dep_avg_balance_3month', 'app_position_type_nm', 'visit_purposes', 'app_vehicle_ind', 'qnt_months_from_last_visit', 'limit_exchange_count']


for column in columns:
    mean_value = data[column].mean()
    data[column].fillna(mean_value, inplace=True)

In [ ]:
label_encoder = LabelEncoder()
data['super_clust'] = label_encoder.fit_transform(data['super_clust'])

In [ ]:
encoding_clust = dict(zip(label_encoder.transform(label_encoder.classes_), label_encoder.classes_))

print("Кодировка clust:")
print(encoding_clust)

Кодировка clust:
{0: 'a. Супер-ЗП (6,15)', 1: 'b. Текущие заемщики (14)', 2: 'c. Масс-ЗП закредитованные (12)', 3: 'd. Масс-ЗП без согласия БКИ (1,3)', 4: 'e. Супер-аффлуент (-1)', 5: 'f. Супер-депозиты (8,16)', 6: 'g. ДК/ЗК до 6 моба (20)', 7: 'h. Бывшие зарплатники (4,9,10,11)', 8: 'i. Бывшие заемщики (2,5,19)', 9: 'j. Депозиты в оттоке (0,17)', 10: 'k. Без продукта (7,13,18)'}


In [ ]:
copy_data = data.copy(deep=True)

In [ ]:
X = copy_data.drop(['super_clust'], axis=1)
y = data['super_clust']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import lightgbm as lgb

model = lgb.LGBMClassifier()

params = {
    "n_estimators": [3, 5, 10, 50, 100, 200],
    "learning_rate": [0.01, 0.05, 0.1, 0.3],
    "max_depth": [1, 3, 5, 10, 15],
    "subsample": [0.6, 0.8, 1]
}

from sklearn.model_selection import GridSearchCV
model_rs = GridSearchCV(model, params, n_jobs=-1, cv=5)

In [ ]:
model_rs.fit(X_train, y_train) # LightGBM with GridSearchCV fitting

print(model_rs.best_params_)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000213 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 790
[LightGBM] [Info] Number of data points in the train set: 687, number of used features: 24
[LightGBM] [Info] Start training from score -2.703693
[LightGBM] [Info] Start training from score -2.794665
[LightGBM] [Info] Start training from score -2.214846
[LightGBM] [Info] Start training from score -1.712053
[LightGBM] [Info] Start training from score -3.641963
[LightGBM] [Info] Start training from score -3.759746
[LightGBM] [Info] Start training from score -3.699121
[LightGBM] [Info] Start training from score -1.813835
[LightGBM] [Info] Start training from score -2.948815
[LightGBM] [Info] Start training from score -2.921416
[LightGBM] [Info] Start training from score -1.432468
[LightGBM] [Warning] No further splits with positive gai

In [ ]:
pred_1 = model_rs.predict(X_val)

from sklearn.metrics import accuracy_score
accuracy_score(y_val, pred_1)

0.6976744186046512

In [ ]:
!pip install joblib

In [ ]:
import joblib

In [ ]:

joblib.dump(model_rs, 'model_file_name.pkl')

['model_file_name.pkl']

In [ ]:
loaded_model = joblib.load('model_file_name.pkl')

In [ ]:
pred_1 = model_rs.predict(X_val)

from sklearn.metrics import accuracy_score
accuracy_score(y_val, pred_1)


0.6976744186046512

In [ ]:
X_val


,gender,age,reg_region_nm,cnt_tr_all_3m,cnt_tr_top_up_3m,cnt_tr_cash_3m,cnt_tr_buy_3m,cnt_tr_mobile_3m,cnt_tr_oil_3m,cnt_tr_on_card_3m,...,max_outstanding_amount_6m,avg_outstanding_amount_3m,cnt_dep_act,sum_dep_now,avg_dep_avg_balance_1month,max_dep_avg_balance_3month,app_vehicle_ind,app_position_type_nm,visit_purposes,qnt_months_from_last_visit
838,1.0,64.623656,34,15.00000,1.000000,1.000000,6.00000,0.000000,0.000000,6.000000,...,46563.10875,22283.478983,0.000000,548440.138748,87.670000,87.670000,0.087209,0,2,22.000000
703,1.0,51.360215,61,48.54717,8.209644,1.146751,19.08805,0.909853,1.157233,14.213836,...,46563.10875,22283.478983,0.408922,548440.138748,537515.472321,355317.681358,0.087209,0,14,8.893333
135,-1.0,36.553763,59,48.54717,8.209644,1.146751,19.08805,0.909853,1.157233,14.213836,...,46563.10875,22283.478983,0.000000,548440.138748,537515.472321,0.084783,0.087209,20,14,8.893333
235,0.0,49.290323,41,48.54717,8.209644,1.146751,19.08805,0.909853,1.157233,14.213836,...,46563.10875,22283.478983,0.408922,548440.138748,537515.472321,355317.681358,0.000000,0,14,8.893333
436,0.0,47.545699,38,48.54717,8.209644,1.146751,19.08805,0.909853,1.157233,14.213836,...,46563.10875,22283.478983,0.408922,548440.138748,537515.472321,355317.681358,0.000000,9,14,8.893333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,-1.0,40.728495,36,48.54717,8.209644,1.146751,19.08805,0.909853,1.157233,14.213836,...,46563.10875,22283.478983,0.000000,0.000000,537515.472321,0.000000,0.087209,20,14,8.893333
267,0.0,44.209677,75,48.54717,8.209644,1.146751,19.08805,0.909853,1.157233,14.213836,...,46563.10875,22283.478983,0.000000,548440.138748,14033.970000,13983.789783,0.000000,4,14,8.893333
373,1.0,66.118280,12,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,46563.10875,22283.478983,0.000000,0.000000,537515.472321,0.000000,0.000000,0,14,8.893333
243,0.0,31.752688,66,48.54717,8.209644,1.146751,19.08805,0.909853,1.157233,14.213836,...,46563.10875,22283.478983,0.408922,548440.138748,537515.472321,355317.681358,0.000000,0,14,8.893333


In [ ]:

pred_dict = {}
pred_1 = model_rs.predict(X_val)

for index, prediction in zip(X_val.index, pred_1):
    result = []
    prediction = encoding_clust[prediction]
    result.extend([index, prediction, X_val.loc[index, 'sum_zp_12m'], X_val.loc[index, 'age'], X_val.loc[index, 'qnt_months_from_last_visit']])
    pred_dict[index] = result